In [37]:
import io
import tempfile
import os
import pandas as pd
import re

In [38]:
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage

In [39]:
import gspread
import subprocess

import PyPDF2
import requests

In [16]:
df = pd.read_excel('../Data/Final_Thawra.xlsx', header=1, index_col='recordnumber')
df.head(2)

,Initials,datecoded,narrative,nameofevent,Startdate,endate,ongoing,timeofprotest,time_range,governorate,...,pagenum1_restored,URL1_restored,Unnamed: 158,drive_text,Unnamed: 160,drive,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165
recordnumber,,,,,,,,,,,,,,,,,,,,,
20190001.0,HAM,2022-11-23 00:00:00,CGTL and Sabaa Party organize a nation wide st...,NaN,2019-01-04,NaT,0.0,NaN,Unspecified,National,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190002.0,HAM,2023-01-12 00:00:00,اهالي تمنين التحتا يقطعون الطريق على تقاطع الم...,NaN,2019-01-04,NaT,NaN,NaN,Unspecified,Beqaa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# df['drive'] = ''
# for _, row in df.iterrows():
#     for i in row.index:
#         if str(row[i]).startswith('https://drive.google'):
#             df.at[_,'drive'] = row[i]

In [41]:
def extract_text_from_pdf(pdf_content):
  resource_manager = PDFResourceManager()
  fake_file_handle = io.StringIO()
  converter = TextConverter(resource_manager, fake_file_handle)
  page_interpreter = PDFPageInterpreter(resource_manager, converter)

  with io.BytesIO(pdf_content) as fh:
    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  check_extractable=True):
      page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

  # close open handles
  converter.close()
  fake_file_handle.close()

  return text

In [42]:
def extract_text_from_pdf(pdf_path):
    # fh = io.BytesIO()
    # downloader = MediaIoBaseDownload(fh, response)
    # pdf_content = fh.getvalue()
    with open(pdf_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            for page_num in range(reader.numPages):
                page = reader.getPage(page_num)
                text += page.extractText()
    return text


In [43]:
def extract_file_id(url):
    pattern = r"open\?id=([\w-]+)"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    else:
        pattern = r"file/d/([\w-]+)"
        match = re.search(pattern, url)
        if match:
            return match.group(1)
        else:
            return None


In [44]:
def clean_text(pdf_text):
    
    #cleaned_text = re.sub(r'[^\u0600-\u06FF\s]', '', pdf_text)
    
    #cleaned_text = re.sub(r'[^؀-ۿ\s]', '', pdf_text)
    cleaned_text = re.sub(r'http\S+', '', pdf_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    cleaned_text = re.sub(r'%[0-9a-fA-F]{2}', '', cleaned_text)
    cleaned_text = cleaned_text.replace('\n',' ').replace('ـ', '').replace('�', '')
    return cleaned_text

In [45]:
# Set up Google Sheets API credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds_sheet = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json', scope)
client = gspread.authorize(creds_sheet)

# Fetch PDF URLs from Google Sheets document
spreadsheet_key = '1xyQe3vgYKGl_hcPa1MOEn970xjTUirEUq_ewfiGjLRs'
sheet_name = 'Main'
worksheet = client.open_by_key(spreadsheet_key).worksheet(sheet_name)


In [46]:
creds_drive = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json',
                                                         scopes=['https://www.googleapis.com/auth/drive'])
service_drive = build('drive', 'v3', credentials=creds_drive)

In [47]:
urls = worksheet.col_values(161)
urls = [x for x in urls if len(x) > 6] 
len(urls)

598

In [48]:
urls = [x for x in urls if x]
len(urls)

598

In [12]:
urls

['https://drive.google.com/file/d/1VqmneDgmkNDJqQ-GHykuN0hyK-XkfiTl/view?usp=share_link',
 'https://drive.google.com/file/d/1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK/view?usp=sharing',
 'https://drive.google.com/open?id=14O_oP08uaJr3MFasdxyjRogFJyuNlTIE&authuser=amrdukmak%40gmail.com&usp=drive_fs',
 'https://drive.google.com/file/d/1VqmneDgmkNDJqQ-GHykuN0hyK-XkfiTl/view?usp=share_link',
 'https://drive.google.com/file/d/1YH5c4v-A2aeQOXebjogFXbX6OPTTvSGN/view?usp=share_link',
 'https://drive.google.com/open?id=14O_oP08uaJr3MFasdxyjRogFJyuNlTIE&authuser=amrdukmak%40gmail.com&usp=drive_fs',
 'https://drive.google.com/file/d/1HeDda9ZmSUn3BbpjoPna1uGrRr7L-fY2/view?usp=share_link',
 'https://drive.google.com/file/d/1HeDda9ZmSUn3BbpjoPna1uGrRr7L-fY2/view?usp=share_link',
 'https://drive.google.com/file/d/1ApbJbabRmpBCoAotIwx04Qb5IazQBIFT/view?usp=sharing',
 'https://drive.google.com/file/d/1HeDda9ZmSUn3BbpjoPna1uGrRr7L-fY2/view?usp=share_link',
 'https://drive.google.com/file/d/1HeDda9ZmSUn3BbpjoPna1

In [49]:
urls = list(dict.fromkeys(urls))
len(urls)

304

In [50]:
bad_urls =[]

In [53]:
count = 0
for url in urls:
    file_id = extract_file_id(url)
    if file_id:
        
        try:
            request = service_drive.files().get_media(fileId=file_id)
            fh = io.BytesIO()
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
                #print("Download %d%%." % int(status.progress() * 100))
            pdf_content = fh.getvalue()
            with open('new.pdf', 'wb') as f:
                f.write(pdf_content)

            pdf = PyPDF2.PdfReader(open('new.pdf', 'rb'))
            
            pdf_text = " "
            for page in pdf.pdf._get_num_pages():
                text = pdf.pages[page].extract_text()
                cleaned_text = clean_text(pdf_text)
                pdf_text.join(cleaned_text)
            cell = worksheet.find(url)
            worksheet.update_cell(cell.row, 163, pdf_text.strip())
            os.remove('new.pdf')
            count += 1
            print((count * 100)/len(urls) , url)
        except:
            print('error')
            bad_urls.append(url)
            cell = worksheet.find(url)
            worksheet.update_cell(cell.row, 163, 'error')
    else:
        cell = worksheet.find(url)
        worksheet.update_cell(cell.row, 163, 'NOT_FOUND')

error
error
error
error
error


In [23]:
for url in bad_urls:
    file_id = extract_file_id(url)
    print(file_id)
    if file_id:
        try:
            request = service_drive.files().get_media(fileId=file_id)
            fh = io.BytesIO()
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
                #print("Download %d%%." % int(status.progress() * 100))
            pdf_content = fh.getvalue()
            with open('new.pdf', 'wb') as f:
                f.write(pdf_content)

            pdf = PyPDF2.PdfReader(open('new.pdf', 'rb'))
            pdf_text = ""

            pdf_text = pdf.pages[0].extract_text()
            text = clean_text(pdf_text)
            cell = worksheet.find(url)
            worksheet.update_cell(cell.row, 167, text)
            os.remove('new.pdf')
            count += 1
            print((count * 100)/len(urls) , url)
        except:
            print('error')
            bad_urls.append(url)
            worksheet.update_cell(cell.row, 167, 'error')
    else:
        cell = worksheet.find(url)
        worksheet.update_cell(cell.row, 167, 'NOT_FOUND')

1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error
1Xa737zv-OIwVrsm-Mwil_yqMMe8pZebK
error


In [94]:
url = 'https://drive.google.com/open?id=12wiwto6X1NpzNYBDjqnxdsuw9YjE-7Cb&authuser=amrdukmak%40gmail.com&usp=drive_fs'

file_id = extract_file_id(url)
print(url)
print(file_id)


https://drive.google.com/open?id=12wiwto6X1NpzNYBDjqnxdsuw9YjE-7Cb&authuser=amrdukmak%40gmail.com&usp=drive_fs
12wiwto6X1NpzNYBDjqnxdsuw9YjE-7Cb


In [ ]:
for url in urls:
    pdf_text = subprocess.check_output(['python', 'pdf_text_extractor.py', url])
    # Update Google Sheets with the extracted text
    cell = worksheet.find(url)
    worksheet.update_cell(cell.row, cell.col -2, pdf_text)

In [156]:
# file_id = extract_file_id(urls[444])
# results = service_drive.files().get(fileId=file_id).execute()

# request = service_drive.files().export_media(fileId=results.get('id'), mimeType=results.get('mimeType'))

#request = service_drive.files().get_media(fileId=file_id)
# fh = io.BytesIO()
# downloader = MediaIoBaseDownload(fh, request)
# #print(request.headers)
# with open('new.pdf', 'wb') as f:
#     f.write(fh.getvalue())

# pdf = PyPDF2.PdfReader(open('new.pdf', 'rb'))
# pdf_text = ""

# for page in range(pdf.getNumPages()):
#     pdf_text += pdf.getPage(page).extractText()
# print(pdf_text)

In [44]:
# response = requests.get(urls[300])
# print(response.headers)

#fh = io.BytesIO(response.content)
#downloader = MediaIoBaseDownload(fh, response.content)
    
# pdf_content = fh.read()
# pdf_content
# with open('new.pdf', 'wb') as f:
#     f.write(pdf_content)
    

# # pdf_content = response.content
# print(pdf_content)
# # with open('new.pdf', 'wb') as f:
# #     f.write(pdf_content)


# pdf = PyPDF2.PdfReader(open('new.pdf', 'rb'))
# pdf_text = ""
# for page in range(pdf.getNumPages()):
#     pdf_text += pdf.getPage(page).extractText()

# # # Print the extracted text
# print(pdf_text)

# # fh = io.BytesIO()
# # downloader = MediaIoBaseDownload(fh, pdf_content)
# # pdf_content = fh.getvalue()
# # with open('new.pdf', 'wb') as f:
# #         f.write(pdf_content)

# # pdf = PyPDF2.PdfReader(open('new.pdf', 'rb'))
# # pdf_text = extract_text_from_pdf('new.pdf')
# # pdf_text
# # # temp_file_path

{'Content-Type': 'text/html; charset=utf-8', 'X-Frame-Options': 'DENY', 'Set-Cookie': '__Host-GAPS=1:OF5YUw7aq7ru6OsDmA2cnNg3FLJtNw:fcsCCvoj9_ntzmWC; Expires=Thu, 22-May-2025 12:59:42 GMT; Path=/; Secure; HttpOnly; Priority=HIGH', 'Vary': 'Sec-Fetch-Dest, Sec-Fetch-Mode, Sec-Fetch-Site', 'x-auto-login': 'realm=com.google&args=service%3Dwise%26continue%3Dhttps://drive.google.com/open?id%253D11kggumhANH1lY4tObWkvCeRYupIpJGIu%2526authuser%253Damrdukmak@gmail.com%2526usp%253Ddrive_fs', 'Link': '<https://www.google.com/intl/en-US/drive/>; rel="canonical"', 'x-ua-compatible': 'IE=edge', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': 'Mon, 01 Jan 1990 00:00:00 GMT', 'Date': 'Tue, 23 May 2023 12:59:42 GMT', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'Content-Security-Policy': "script-src 'nonce-LOruTlHobS3HJdI2LybHaQ' 'unsafe-inline';object-src 'none';base-uri 'self';report-uri /v3/signin/_/AccountsSignInUi/cspreport

In [ ]:
# Extract text from each PDF using the local Python function
